# Chapter 3: Convolutional Neural Networks
You'll need to either copy the image training files you downloaded in Chapter 2 to this directory, or alter the paths appropriately.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## CNNNet (or AlexNet by another name…)

In [6]:
import torch
import torch.nn.functional as F

# Define a 4x4 input image tensor with one channel
input_image = torch.tensor([[1, 2, 3, 4],
                            [5, 6, 7, 8],
                            [9, 10, 11, 12],
                            [13, 14, 15, 16]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Batch and channel dimensions

# Define a custom 3x3 kernel
kernel = torch.tensor([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Batch and channel dimensions

# Apply 2D convolution using the custom kernel
output = F.conv2d(input_image, kernel, padding=0)

# Print the input, kernel, and output tensors
print("Input Image:")
print(input_image)
print(torch.flatten(input_image),1)
print("Custom Kernel:")
print(kernel)
print("Convolution Output:")
print(output)


Input Image:
tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 16.]) 1
Custom Kernel:
tensor([[[[ 1.,  0., -1.],
          [ 2.,  0., -2.],
          [ 1.,  0., -1.]]]])
Convolution Output:
tensor([[[[-8., -8.],
          [-8., -8.]]]])


In [7]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [8]:
cnnnet = CNNNet()

In [9]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [10]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "../train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "../val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [11]:
device

device(type='cuda')

In [12]:
import os
os.getcwd()

'C:\\Users\\saad.naeem\\PycharmProjects\\beginners-pytorch-deep-learning\\chapter3'

In [13]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [14]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 0.96, Validation Loss: 0.65, accuracy = 0.84
Epoch: 2, Training Loss: 0.69, Validation Loss: 0.63, accuracy = 0.84
Epoch: 3, Training Loss: 0.63, Validation Loss: 0.59, accuracy = 0.65
Epoch: 4, Training Loss: 0.55, Validation Loss: 0.41, accuracy = 0.80
Epoch: 5, Training Loss: 0.46, Validation Loss: 0.55, accuracy = 0.71
Epoch: 6, Training Loss: 0.43, Validation Loss: 0.68, accuracy = 0.65
Epoch: 7, Training Loss: 0.44, Validation Loss: 0.46, accuracy = 0.80
Epoch: 8, Training Loss: 0.39, Validation Loss: 0.43, accuracy = 0.82
Epoch: 9, Training Loss: 0.36, Validation Loss: 0.36, accuracy = 0.82
Epoch: 10, Training Loss: 0.34, Validation Loss: 0.56, accuracy = 0.75


## Downloading a pretrained network 

There are two ways of downloading pre-trained image models with PyTorch. Firstly, you can use the `torchvision.models` library, or you can use PyTorch Hub. The latter is preferred as of 2019, as this is a one-stop shop for all models and the new standard for distributing models with PyTorch.

In [15]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=1000, pretrained=True)

C:\Users\saad.naeem\AppData\Local\anaconda3\envs\pytorch-deep-learning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\saad.naeem\AppData\Local\anaconda3\envs\pytorch-deep-learning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\saad.naeem/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:34<00:00, 7.07MB/s] 


In [16]:
resnet50 = torch.hub.load('pytorch/vision', 'resnet50')

Downloading: "https://github.com/pytorch/vision/zipball/main" to C:\Users\saad.naeem/.cache\torch\hub\main.zip


In [17]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [2]:
# !pip install textblob
from textblob import TextBlob

In [7]:
# Your input sentence
sentence = "although the restaurant ambience was great, but the food was not good."

# Create a TextBlob object
blob = TextBlob(sentence)

# Perform sentiment analysis
sentiment = blob.sentiment

# Print the sentiment polarity and subjectivity
print("Sentiment Polarity:", sentiment.polarity)
print("Sentiment Subjectivity:", sentiment.subjectivity)

# Interpret the sentiment
if sentiment.polarity > 0:
    print("The sentiment is positive.")
elif sentiment.polarity < 0:
    print("The sentiment is negative.")
else:
    print("The sentiment is neutral.")


Sentiment Polarity: 0.22500000000000003
Sentiment Subjectivity: 0.675
The sentiment is positive.


In [13]:
from textblob import TextBlob

# Your input sentence
sentence = "although the restaurant ambience was good, but the food was bad."

# Create a TextBlob object
blob = TextBlob(sentence)

# Initialize sentiment variables
total_polarity = 0
total_subjectivity = 0

# Analyze the sentiment for each clause
for clause in sentence.split(', '):
    clause_blob = TextBlob(clause)
    total_polarity += clause_blob.sentiment.polarity
    total_subjectivity += clause_blob.sentiment.subjectivity

# Calculate the overall sentiment
overall_sentiment = total_polarity / len(sentence.split(', '))

# Print the overall sentiment and subjectivity
print("Overall Sentiment Polarity:", overall_sentiment)
print("Overall Sentiment Subjectivity:", total_subjectivity)

# Interpret the overall sentiment
if overall_sentiment > 0:
    print("The overall sentiment is positive.")
elif overall_sentiment < 0:
    print("The overall sentiment is negative.")
else:
    print("The overall sentiment is neutral.")


Overall Sentiment Polarity: 5.551115123125783e-17
Overall Sentiment Subjectivity: 1.2666666666666666
The overall sentiment is positive.


In [15]:
# !pip install nltk

In [16]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saad.naeem\AppData\Roaming\nltk_data...


True

In [21]:
from nltk.tokenize import word_tokenize

sentence = "although the restaurant ambience was good, but the food was bad."
words = word_tokenize(sentence)


In [22]:
# nltk.download('punkt')

In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Get the sentiment scores for the sentence
sentiment_scores = sia.polarity_scores(sentence)

# Determine sentiment based on compound score
compound_score = sentiment_scores['compound']

if compound_score >= 0.05:
    sentiment = "Positive"
elif compound_score <= -0.05:
    sentiment = "Negative"
else:
    sentiment = "Neutral"


In [24]:
print("Sentence: ", sentence)
print("Sentiment: ", sentiment)


Sentence:  although the restaurant ambience was good, but the food was bad.
Sentiment:  Negative


In [28]:
# !pip install --force-reinstall transformers
!pip install sentencepiece


  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load the T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define the input sentence and the target class
input_sentence = "I am calling because I am facing issues with my internet connection."
target_class = "orange"

# Prepare the input data in a text-to-text format
input_text = f"extract whether the class '{target_class}' exists in the sentence: {input_sentence}"

# Tokenize the input data
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate output text
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the output
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", output_text)

# Check if the target class is present in the output
class_exists = target_class in output_text.lower()
print(f"Target class '{target_class}' exists in the sentence: {class_exists}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Output: True
Target class 'orange' exists in the sentence: False
